## Interpretable ML for Thermal Optimality Analysis

### Arman Ahmadi

https://doi.org/10.1029/2024JH000445

### Setup

#### Importing Modules and Libraries

In [ ]:
# --- Notebook setup: paths, imports, seeds ---
from pathlib import Path
import os, random
import numpy as np
import pandas as pd

# Paths (repo-relative)
ROOT = Path(__file__).resolve().parents[1] if\
'__file__' in globals() else\
      Path.cwd().resolve().parent
DATA_DIR = ROOT / 'data' / 'sample'
OUT_DIR = ROOT / 'outputs'
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Core plotting
import matplotlib.pyplot as plt

# ML
from sklearn.metrics import mean_absolute_error, r2_score

# Remove warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Standard Libraries
from datetime import datetime # time formatting
from tqdm.notebook import tqdm # loops
import glob

# Data Science Libraries
import sklearn as skl

# Visualization Libraries
import seaborn as sns

### Data Importing and Preprocessing

#### Import Data

In [ ]:
path = DATA_DIR / 'sites'

# Make a list of all CSV files in the path
all_files = glob.glob(str(path / '*.csv'))

# Make a list of pd DataFrames
sites = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)

    # Extract SiteID from the filename (assuming the format is "AMF_XX-XXX_...")
    try:
        site_id = Path(filename).stem.split('_')[1]
    except IndexError:
        site_id = 'UNKNOWN'

    # Add a new column "SiteID" with the extracted value
    df['SiteID'] = site_id

    sites.append(df)

In [ ]:
# Concatenate all DFs into one

Allsites1 = pd.concat(sites, axis=0, ignore_index=True)

In [ ]:
print('Number of daily samples:', len(Allsites1))

#### Selecting and Renaming Features

In [ ]:
# Select columns

Allsites2 = Allsites1[['TIMESTAMP', 'SiteID', 'TA_F', 'SW_IN_F', 'LW_IN_F', 'LW_OUT', 'VPD_F', 'P_F',
                           'WS_F', 'CO2_F_MDS', 'LE_F_MDS', 'H_F_MDS',
                              'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF']]

In [ ]:
# Rename columns

Allsites2.columns = ['TimeStamp', 'SiteID', 'T_air', 'SW', 'LW_In', 'LW_Out', 'VPD', 'precipitation',
                        'WS', 'CO2', 'LE', 'H',
                           'GPP_night', 'GPP_day']

#### Growing Season Data Only (May-Sep)

In [ ]:
# Format the 'TimeStamp' column to datetime object

Allsites2['TimeStamp'] = pd.to_datetime(Allsites2['TimeStamp'], format='%Y%m%d') # CSV format is YYYYMMDD

In [ ]:
# May-Sep mask

mask = (Allsites2['TimeStamp'].dt.month >= 5) & (Allsites2['TimeStamp'].dt.month <= 9)

In [ ]:
# Allsites2: Full (year-round) data
# Allsites3: Growing season data

Allsites3 = Allsites2[mask]

In [ ]:
# # Only for considering the full year data

# Allsites3 = Allsites2

#### Removing Unexpected Values

In [ ]:
print('Number of samples before removing rows containing NAN or negative/zero values:', len(Allsites3))

In [ ]:
# Remove samples with NAN for LE or GPP

Allsites3 = Allsites3.dropna(subset = ['LE', 'GPP_day', 'GPP_night'])


# Remove samples with negative/zero values for LE or GPP

Allsites3 = Allsites3[(~Allsites3[['LE', 'GPP_day', 'GPP_night']].lt(0).any(axis=1)) &
                    (~Allsites3[['LE', 'GPP_day', 'GPP_night']].eq(0).any(axis=1))]

In [ ]:
print('Number of samples after removing rows containing NAN or negative/zero values for LE and GPP:', len(Allsites3))

In [ ]:
# Remove samples with negative/zero values for T_air

Allsites3 = Allsites3[(~Allsites3[['T_air']].lt(0).any(axis=1)) &
                      (~Allsites3[['T_air']].eq(0).any(axis=1))]

In [ ]:
print('Number of samples after removing rows containing negative/zero values for T_air:', len(Allsites3))

In [ ]:
# Remove samples with negative/zero values for CO2 concentration

Allsites3 = Allsites3[(~Allsites3[['CO2']].lt(0).any(axis=1)) &
                    (~Allsites3[['CO2']].eq(0).any(axis=1))]

In [ ]:
print('Number of samples after removing zero and negative CO2 values:', len(Allsites3))

In [ ]:
# Remove samples with unrealistically high level of CO2 concentration

Allsites3 = Allsites3[Allsites3.CO2 < 700]

In [ ]:
print('Number of samples after removing unexpectedly high values of CO2:', len(Allsites3))

#### Changing Units and Adding Features

In [ ]:
# Change the units of SW radiation from W.m-2 to MJ.m-2.day-1

Allsites3['SW_MJ'] = Allsites3.SW * ((3600*24)/1000000)

In [ ]:
# Add evaporative fraction

Allsites3['EF'] = Allsites3.LE / (Allsites3.LE + Allsites3.H)

#### Adding Site-level Features from Ameriflux_191

In [ ]:
Ameriflux191 = pd.read_csv(DATA_DIR/'Ameriflux_fluxnet_list_191.csv', index_col=None, header=0)

In [ ]:
# Merge the two DataFrames based on 'SiteID'

merging_columns = ['SiteID','IGBP', 'Koppen', 'MAP',
                  'MAT', 'Country', 'Latitude', 'Longitude',
                  'Elevation', 'Wetness_Index']

Allsites = pd.merge(Allsites3, Ameriflux191[merging_columns], on='SiteID', how='left')

#### Combining IGBP and Climate Classes

IGBP:

- WSA and SAV combined and called SAV

- CSH and OSH combined and called SHR

Climate:

- Humid Continental (HC): Dfb, Dfa, Dsb, Dsa

- Humid Subtropical (HS): Cfa

- Subarctic (S): Dfc, Dfd, Dwc

- Mediterranean (M): Csa, Csb

- Semi-Arid (SA): Bsk, Bsh

- Tundra (T): ET

- Oceanic (O): Cfb

- Arid (A): BWk, BWh

In [ ]:
Allsites['PFT'] = Allsites['IGBP'].replace({'WSA': 'SAV', 'CSH': 'SHR', 'OSH': 'SHR'})

In [ ]:
Allsites['PFT'].unique()

In [ ]:
Allsites['Climate'] = Allsites['Koppen'].replace({'Dfb': 'HC', 'Dfa': 'HC', 'Dsb': 'HC', 'Dsa': 'HC',
                                                  'Cfa': 'HS',
                                                  'Dfc': 'S', 'Dfd': 'S', 'Dwc': 'S',
                                                  'Csa': 'M', 'Csb': 'M',
                                                  'Bsk': 'SA', 'Bsh': 'SA',
                                                  'ET': 'T',
                                                  'Cfb': 'O',
                                                  'Bwk': 'A', 'Bwh': 'A'})

In [ ]:
Allsites['Climate'].unique()

In [ ]:
# Remove samples with negative EF

Allsites = Allsites[Allsites.EF > 0]

In [ ]:
# Remove samples with EF values higher than 1.4

Allsites = Allsites[Allsites.EF <= 1.4]

In [ ]:
print('Number of samples after removing bad EF values:', len(Allsites))

### Data Analysis and Visualization


#### GPP Distribution (BoxPlots)

In [ ]:
# Make a figure (canvas) object and an axes (inside canvas) object
f, axes = plt.subplots(1,1, figsize=(6,3))

desired_order = ['ENF', 'DBF', 'SAV', 'SHR', 'GRA', 'WET', 'CRO']  # PFT order
# desired_order = ['HC', 'HS', 'M', 'S', 'T', 'SA']  # Climate order

# Define a color palette
palette_7 = [
    '#1f77b4',  # Blue
    '#ff7f0e',  # Orange
    '#2ca02c',  # Green
    '#d62728',  # Red
    '#9467bd',  # Purple
    '#8c564b',  # Brown
    '#e377c2',  # Pink
]


z1 = sns.boxplot(data = Allsites, x = 'PFT', y = 'GPP_day', order = desired_order, palette=palette_7,
                 linewidth=1.3, showfliers = False, width=0.5)

z1.axes.set_ylabel('GPP (gC/m^2/day)')

sns.set(font_scale=1.2,style='whitegrid')
plt.tight_layout()

# plt.savefig('GPP_boxplots_PFT.jpg', dpi=400)

#### Meteorological Variables Distribution (Boxplots)

In [ ]:
# Define a color palette
palette_7 = [
    '#1f77b4',  # Blue
    '#ff7f0e',  # Orange
    '#2ca02c',  # Green
    '#d62728',  # Red
    '#9467bd',  # Purple
    '#8c564b',  # Brown
    '#e377c2',  # Pink
]

f, axes = plt.subplots(4,1, figsize=(6,10))

desired_order = ['ENF', 'DBF', 'SAV', 'SHR', 'GRA', 'WET', 'CRO']  # PFT order
# desired_order = ['HC', 'HS', 'M', 'S', 'T', 'SA']  # Climate order


# List of y-axis variables in the order of subplot axes
y_vars = ['T_air', 'CO2', 'SW', 'EF']

# List of plt axes
zplots = []

for i, y_var in enumerate(y_vars):
    z = sns.boxplot(
        data = Allsites,
        x = 'PFT',
        y = y_var,
        order = desired_order,
        linewidth = 1.3,
        ax = axes[i],
        palette = palette_7,
        showfliers = False,
        width = 0.5
    )
    zplots.append(z)


for i in range(len(zplots)-1):
  zplots[i].axes.get_xaxis().set_visible(False)

zplots[0].axes.set_ylabel('T_air (deg. C)')
zplots[1].axes.set_ylabel('CO2 (ppm)')
zplots[2].axes.set_ylabel('SW (W/m^2)')
zplots[3].axes.set_ylabel('EF')

sns.set(font_scale=1.2,style='whitegrid')
plt.tight_layout()

# plt.savefig('PFT_boxplots.jpg', dpi=400)

#### The Number of Samples and Sites

In [ ]:
Input_Samples_PFT = Allsites.groupby('PFT').agg(Samples=('PFT', 'size'), Sites=('SiteID', 'nunique')).reset_index()

desired_order = ['ENF', 'DBF', 'SAV', 'SHR', 'GRA', 'WET', 'CRO']

# Make PFT column categorical with a desired order
Input_Samples_PFT['PFT'] = pd.Categorical(Input_Samples_PFT['PFT'], categories=desired_order, ordered=True)

# Sort the rows according to the PFT desired order
Input_Samples_PFT = Input_Samples_PFT.sort_values(by='PFT')

In [ ]:
Input_Samples_PFT

In [ ]:
Input_Samples_Climate = Allsites.groupby('Climate').agg(Samples=('Climate', 'size'), Sites=('SiteID', 'nunique')).reset_index()

desired_order = ['HC', 'HS', 'M', 'S', 'T', 'SA' ]

Input_Samples_Climate['Climate'] = pd.Categorical(Input_Samples_Climate['Climate'], categories=desired_order, ordered=True)

Input_Samples_Climate = Input_Samples_Climate.sort_values(by='Climate')

In [ ]:
Input_Samples_Climate

### Interpretable ML

#### Packages and Libraries

In [ ]:
# Required libraries

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from math import sqrt
from sklearn.inspection import PartialDependenceDisplay # PDP plots
import optuna
from PyALE import ale

#### XGBoost Model

In [ ]:
# Select data

data = Allsites[Allsites.PFT == 'WET']
# data = Allsites[(Allsites.Wetness_Index  > 0.6) & (Allsites.Wetness_Index  <= 0.8)]

In [ ]:
# Correlation Matrix

data_x = data[['T_air', 'CO2', 'SW', 'EF']]

# Calculate the correlation matrix
correlation_matrix = data_x.corr('pearson')

# Create a heatmap using seaborn
plt.figure(figsize=(3.5, 3))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, center=0)
plt.title('WET')

plt.tick_params(bottom=True, left=True)
sns.set(font_scale=1, style='white')
plt.tight_layout()

# plt.savefig('correlation_matrix_WET.jpg', dpi=400)

In [ ]:
# Training and testing the model

# Split the data into features (X) and target variable (y)
X = data[['T_air', 'CO2', 'SW', 'EF']]
y = data['GPP_day']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the XGBoost regressor
model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance (using Root Mean Squared Error)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse} gC/m2/day')

r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

In [ ]:
# Hyperparameter Optimization with Optuna

# Split the data into features (X) and target variable (y)
X = data[['T_air', 'CO2', 'SW', 'EF']]
y = data['GPP_day']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']), # categorical parameter
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0), # continuous parameter
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10), # integer parameter
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0), # floating point parameter
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        # 'tree_method': 'gpu_hist'  # Enable GPU
    }

    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    return rmse

# Create and run the Optuna study with a TPE sampler and Median Pruner
sampler = optuna.samplers.TPESampler(seed=42) # TPE (Tree-structured Parzen Estimator) algorithm
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10, interval_steps=1)
study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=20) # n_tirals=100 for real applications

# Print the best hyperparameters and corresponding RMSE
print("Best Hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Store the best set of hyperparameters
best_params = study.best_params

In [ ]:
best_params

In [ ]:
# Training the model with the best hyperparameters

# Split the data into features (X) and target variable (y)
X = data[['T_air', 'CO2', 'SW', 'EF']]
y = data['GPP_day']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the XGBoost regressor with the best hyperparameters
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **best_params, importance_type='gain')

# Train the model on the entire training set
model.fit(X_train, y_train)

# Get feature importances
feature_importance = model.feature_importances_

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance (using Root Mean Squared Error)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse} gC/m2/day')

r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

# Print feature importance
print()
print("Feature Importance:")
for i, feature in enumerate(X.columns):
    print(f"{feature}: {feature_importance[i]}")

In [ ]:
# Create scatter plot with 1:1 line

fig, ax = plt.subplots(figsize=(5, 4))

plt.scatter(y_test, y_pred, alpha=0.5, s=20, c='b')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red', linewidth=2, label='1:1 line')

# Add labels and title
plt.xlabel('Actual GPP (gC/m2/day)')
plt.ylabel('Predicted GPP (gC/m2/day)')
plt.title('WET')

# Add R-squared and RMSE annotations
plt.annotate(f'R-squared: {r2:.3f}', xy=(0.05, 0.85), xycoords='axes fraction', fontsize=10)
plt.annotate(f'RMSE: {rmse:.3f} gC/m2/day', xy=(0.05, 0.78), xycoords='axes fraction', fontsize=10)

# Add legend
plt.legend(loc=(0.05, 0.90))

plt.tick_params(bottom=True, left=True)
plt.tight_layout()


# plt.savefig('Scatterplot_WET.jpg', dpi=400)

#### PDP and ALE Plots

In [ ]:
# 2-D PDP plts

fig, ax = plt.subplots(figsize=(4.5, 3.5))
PartialDependenceDisplay.from_estimator(estimator=model, X=X, features=[0], ax=ax, grid_resolution=20)
plt.title('WET')

plt.tick_params(bottom=True, left=True)
plt.tight_layout()

# plt.savefig('PDP2D_WET.jpg', dpi=400)

In [ ]:
# 3-D PDP plots

fig, ax = plt.subplots(figsize=(4.5, 3.5))
features = [(0, 3)]  # Choose the indices of the two features for the 2D plot
PartialDependenceDisplay.from_estimator(estimator=model, X=X_train, features=features, ax=ax, grid_resolution=10)
ax.set_title('WET')


plt.tick_params(bottom=True, left=True)
plt.tight_layout()

plt.set_cmap('viridis')

# plt.savefig('PDP3D_WET.jpg', dpi=400)

In [ ]:
# 2-D ALE plot

fig, ax = plt.subplots(figsize=(4.5, 3.5))
ale_eff = ale(X=X_train, model=model, feature=['T_air'], grid_size=20, include_CI=False, fig=fig, ax=ax)

ax.set_title('WET')
ax.set_xlabel("T_air")
ax.set_ylabel("Effect on Prediction (centered)")

plt.tick_params(bottom=True, left=True)
plt.tight_layout()

# plt.savefig('ALE_WET.jpg', dpi=400)

#### Feature Importance Visualizations

In [ ]:
FI_PFT = pd.read_csv(DATA_DIR/'FeatureImportance_PFT.csv', index_col=None, header=0)
FI_Climate = pd.read_csv(DATA_DIR/'FeatureImportance_Climate.csv', index_col=None, header=0)
FI_WI = pd.read_csv(DATA_DIR/'FeatureImportance_WetnessIndex.csv', index_col=None, header=0)

In [ ]:
FI_WI

In [ ]:
df = FI_WI

# Define colors for each column
colors = ['lightcoral', 'lightgreen', 'gold', 'lightskyblue']

# Create figure and axes objects
fig, axs = plt.subplots(1, len(df), figsize=(18, 3))

# Iterate through each row of the DataFrame
for i, row in df.iterrows():
    # Extract WI and numeric values for each row
    pft = row['WI']
    values = row[['T_air', 'CO2', 'SW', 'EF']].tolist()

    # Create pie chart for each row
    axs[i].pie(values, labels=['T_air', 'CO2', 'SW', 'EF'], colors=colors, autopct='%1.1f%%', startangle=140)
    axs[i].set_title(pft)

# Adjust layout
plt.tight_layout()


# plt.savefig('Pie_WI.jpg', dpi=400)

### Trend Analysis

#### Data Loading and Preparation

In [ ]:
# Load ERA5 Data

path = DATA_DIR / 'ERA5'
all_files = glob.glob(str(path / '*.csv'))

ERA5 = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)

    # Extract SiteID from the filename (assuming the format is "AMF_XX-XXX_...")
    site_id = filename.split('_')[1]

    # Add a new column "SiteID" with the extracted value
    df['SiteID'] = site_id

    ERA5.append(df)

In [ ]:
# TIMESTAMP to datetime

for i in range(len(ERA5)):
  ERA5[i]['TIMESTAMP'] = pd.to_datetime(ERA5[i]['TIMESTAMP'], format='%Y%m') # YYYYMM

In [ ]:
# Select May to September

for i in range(len(ERA5)):
  mask = (ERA5[i]['TIMESTAMP'].dt.month >= 5) & (ERA5[i]['TIMESTAMP'].dt.month <= 9)
  ERA5[i] = ERA5[i][mask]

In [ ]:
# Select and rename the desired columns

for i in range(len(ERA5)):
  ERA5[i] = ERA5[i][['TIMESTAMP', 'SiteID', 'TA_ERA_DAY']]
  ERA5[i].columns = ['Date', 'SiteID', 'T_air_day']

In [ ]:
# Get the average T_air_day for months 5 to 9 for each year

for i in range(len(ERA5)):
  ERA5[i]['Year'] = ERA5[i]['Date'].dt.year
  ERA5[i] = ERA5[i].groupby(['Year', 'SiteID'])['T_air_day'].mean().reset_index()

In [ ]:
ERA5[0].head()

#### Mann-Kendall Test

In [ ]:
# pymannkendall library

import pymannkendall as mk

In [ ]:
records = []

# Iterate over each DataFrame in ERA5 list
for df in ERA5:
    # Get the SiteID from the DataFrame
    site_id = df['SiteID'].iloc[0] # get the SiteID from the first row

    # Perform Mann-Kendall test
    result = mk.original_test(df['T_air_day'])

    # Extract Z and p-values
    Z = result.z
    p = result.p

    # Append SiteID, Z, and p-values to MK_trend DataFrame
    records.append({'SiteID': site_id, 'Z': Z, 'P': p})

MK_trend = pd.DataFrame(records)

In [ ]:
# Ameriflux191 = pd.read_csv(DATA_DIR/'Ameriflux_fluxnet_list_191.csv', index_col=None, header=0)

In [ ]:
# Merge the two DataFrames based on 'SiteID'

merging_columns = ['SiteID','IGBP', 'Koppen', 'Wetness_Index']
MK_trend = pd.merge(MK_trend, Ameriflux191[merging_columns], on='SiteID', how='left')

In [ ]:
MK_trend['PFT'] = MK_trend['IGBP'].replace({'WSA': 'SAV', 'CSH': 'SHR', 'OSH': 'SHR'})

In [ ]:
MK_trend['Climate'] = MK_trend['Koppen'].replace({'Dfb': 'HC', 'Dfa': 'HC', 'Dsb': 'HC', 'Dsa': 'HC',\
                                                  'Cfa': 'HS', 'Dfc': 'S', 'Dfd': 'S', 'Dwc': 'S',\
                                                  'Csa': 'M', 'Csb': 'M', 'Bsk': 'SA', 'Bsh': 'SA',\
                                                  'ET': 'T', 'Cfb': 'O', 'Bwk': 'A', 'Bwh': 'A'})

In [ ]:
MK_trend.head()

In [ ]:
## Create a new boolean column indicating whether P value is less than 0.05
MK_trend['Significant'] = MK_trend['P'] < 0.05

# Define a custom color palette for significant and non-significant Z values
palette = {True: 'blue', False: 'red'}

# Create the swarmplot
plt.figure(figsize=(7.5, 3))
sns.swarmplot(data=MK_trend, x='PFT', y='Z', hue='Significant', palette=palette, s=6)

plt.legend().remove()

# Add labels and title
plt.xlabel('')
plt.ylabel('Z value')
plt.title('Wetness Index')
plt.grid(axis='y', linestyle='--')

plt.tick_params(bottom=True, left=True)
plt.tight_layout()

# plt.savefig('MKTest_WI.jpg', dpi=400)

### Thermal Acclimation of Photosynthesis

In [ ]:
# Get unique SiteID values as an np array
unique_site_ids = Allsites['SiteID'].unique()

# List to store the results
results = []

# Iterate over each unique SiteID
for site_id in unique_site_ids:

    # Filter the DataFrame for the current SiteID
    subset_df = Allsites[Allsites['SiteID'] == site_id]

    # Sort subset by GPP in descending order
    sorted_subset = subset_df.sort_values(by='GPP_day', ascending=False)

    # Calculate the number of top 10% records
    number_of_records = len(sorted_subset)
    top_10_percent_count = max(1, int(number_of_records * 0.10))

    # Select the top 10% of records
    top_10_percent_df = sorted_subset.head(top_10_percent_count)

    # Calculate the average of T_air for these records
    Opt_T_air = top_10_percent_df['T_air'].mean()

    # Calculate the weighted average of T_air
    W_avg_T_air = (subset_df.T_air * subset_df.GPP_day).sum() / subset_df.GPP_day.sum()

    # Append to results
    results.append({'SiteID': site_id, 'Opt_T_air': Opt_T_air, 'W_avg_T_air': W_avg_T_air})

# Create a new DataFrame from the results
T_air_df = pd.DataFrame(results)

In [ ]:
T_air_df.head()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from scipy.stats import linregress

In [ ]:
rmse = sqrt(mean_squared_error(T_air_df.W_avg_T_air, T_air_df.Opt_T_air))

r2 = r2_score(T_air_df.W_avg_T_air, T_air_df.Opt_T_air)

In [ ]:
# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(T_air_df.W_avg_T_air, T_air_df.Opt_T_air)

# Create scatter plot
fig, ax = plt.subplots(figsize=(5, 4))
plt.scatter(T_air_df.W_avg_T_air, T_air_df.Opt_T_air, alpha=0.7, s=20)

# Plot regression line
x_vals = np.array(ax.get_xlim())
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '--', color='red', linewidth=2, label=f'Regression line\ny = {slope:.2f}X + {intercept:.2f}')

# Add labels and title
plt.xlabel('GPP-weighted average T_air (deg.C)')
plt.ylabel('Optimum T_air (deg.C)')
plt.title('')

# Add R-squared annotation
plt.annotate(f'R²= {r_value**2:.2f}', xy=(0.02, 0.9), xycoords='axes fraction', fontsize=12)

# Add legend
plt.legend()

plt.tight_layout()

# Save or show the plot
# plt.savefig('Optimum_vs_average_T_air.jpg', dpi=400)